# Environment setup

1.   List item
2.   List item



## Run the two cells below for output wrap-around

In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [2]:
from notebook.services.config import ConfigManager
c = ConfigManager()
c.update('notebook', {"CodeCell": {"cm_config": {"lineWrapping": True}}})

{'CodeCell': {'cm_config': {'lineWrapping': True}}}

## Importing libraries

In [3]:
import math
import altair as alt
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from pprint import pprint
import geemap
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
import lightgbm as lgb
import statsmodels.api as sm
from scipy.stats import zscore

## Authentication and Initialization

In [4]:
import ee
ee.Authenticate()

True

In [5]:
ee.Initialize(project='ee-anaad')

## Loading Random Forest (Landsat-8)

In [6]:
random_forest_classifier = ee.Classifier.load('projects/ee-anaad/assets/Classifiers/random_forest_classifier')
def getFireMaskOldRandomForest(image):
    return image.classify(random_forest_classifier).rename(['F'])

# Time Series Analysis

In [7]:
district_boundaries = ee.FeatureCollection('projects/ee-anaad/assets/India_district_boundaries')
aez_boundaries = ee.FeatureCollection('projects/ee-anaad/assets/India_AEZ_boundaries')
block_boundaries = ee.FeatureCollection('projects/ee-anaad/assets/AEZ_mapped_block_boundaries')

In [8]:
# Run the code below to visualize AEZ boundaries
m = geemap.Map()
m.addLayerControl()

#adding blocks belonging to aez 12
blocks = block_boundaries.filter(ee.Filter.eq('aez_id', 12))
m.addLayer(blocks, {}, 'Blocks_12')

#adding aez 12 boundary
aez = aez_boundaries.filter(ee.Filter.eq('ae_regcode', 12))
m.addLayer(aez, {}, 'AEZ_12')

m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Helper Functions

In [9]:
bands_to_select = ['Tair_f_inst', 'Evap_tavg', 'Qair_f_inst', 'Rainf_f_tavg', 'SoilTMP0_10cm_inst', 'Wind_f_inst']
bands_add = ['Evap_tavg', 'Rainf_f_tavg']
bands_avg = ['Tair_f_inst', 'Qair_f_inst', 'SoilTMP0_10cm_inst', 'Wind_f_inst']

def get_data_row(aoi_geo, landsatImagePrefix, date_object, method):
    dict_row = {}

    getFireMask = getFireMaskOldRandomForest

    landsatImageId = landsatImagePrefix+date_object.strftime("%Y%m%d")
    try:
        ee.data.getAsset(landsatImageId)
    except ee.EEException:
        print(landsatImageId, 'does not exist.')
        return None
    image = ee.Image(landsatImageId).clip(aoi_geo)
    fire_mask = getFireMask(image)#.updateMask(forest_mask)
    fire = fire_mask.updateMask(fire_mask)
    fire_pixels = fire.reduceRegion(reducer = ee.Reducer.sum(), scale = 30, maxPixels = 1e8)
    num_fires = ee.Number(fire_pixels.get('F')).getInfo()
    # fire_vectors = fire.addBands(fire).reduceToVectors(
    #     geometry = aoi_buffered,
    #     scale = 30,
    #     reducer = ee.Reducer.mean(),
    #     maxPixels = 1e10
    # )
    # num_fires = fire_vectors.size().getInfo()
    dict_row['Num_fires'] = num_fires

    def get_climatic_data(ind):
        start_date = (date_object+timedelta(days=ind)).strftime("%Y-%m-%d")
        end_date = (date_object+timedelta(days=ind+1)).strftime("%Y-%m-%d")
        ic = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H")\
        .filterBounds(aoi_geo)\
        .filterDate(start_date, end_date)

        image1 = ic.select(bands_avg).mean().clip(aoi_geo)
        mean_values = image1.reduceRegion(reducer=ee.Reducer.mean(), geometry=aoi_geo)
        values = mean_values.getInfo()
        for key in values:
            dict_row[key+str(ind)] = values[key]

        image2 = ic.select(bands_add).sum().clip(aoi_geo)
        mean_values = image2.reduceRegion(reducer=ee.Reducer.mean(), geometry=aoi_geo)
        values = mean_values.getInfo()
        for key in values:
            dict_row[key+str(ind)] = values[key]*86400

    for ind in range(3):
        get_climatic_data(ind)

    return dict_row


def get_data_matrix(aoi, start_date, end_date, method):
    dict_rows = []
    aoi_geo = aoi.geometry()

    landsatImageCollection = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")\
        .filterBounds(aoi_geo)\
        .filterDate(start_date, (datetime.strptime(start_date, "%Y-%m-%d")+timedelta(days=16)).strftime("%Y-%m-%d"))
    while(landsatImageCollection.size().getInfo() == 0):
        start_date = (datetime.strptime(start_date, "%Y-%m-%d")+timedelta(days=16)).strftime("%Y-%m-%d")
        landsatImageCollection = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")\
        .filterBounds(aoi_geo)\
        .filterDate(start_date, (datetime.strptime(start_date, "%Y-%m-%d")+timedelta(days=16)).strftime("%Y-%m-%d"))

    n = landsatImageCollection.size().getInfo()
    landsatImageList = ee.List(landsatImageCollection.toList(n))
    max_intersection_area = 0
    landsatImageId = None
    for i in range(n):
        landsatImage = ee.Image(landsatImageList.get(i))
        intersection_area = landsatImage.geometry().intersection(aoi_geo).area().getInfo()
        if intersection_area > max_intersection_area:
            max_intersection_area = intersection_area
            landsatImageId = landsatImage.id().getInfo()
    date_string = landsatImageId[-8:]
    date_object = datetime.strptime(date_string, "%Y%m%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    landsatImagePrefix = 'LANDSAT/LC08/C02/T1_TOA/'+landsatImageId[:11]+'_'

    adi = aoi.get('ADI 2019').getInfo()
    while date_object < end_date:
        dict_row = get_data_row(aoi_geo, landsatImagePrefix, date_object, method)
        if dict_row != None:
            dict_row['ADI'] = adi
            dict_rows.append(dict_row)
        date_object = date_object+timedelta(days=16)

    return pd.DataFrame(dict_rows)

# Use like this
# block = block_boundaries.filter(ee.Filter.eq('Subdistric', 'Masalia')).first()
# block_data = get_data_matrix(block, '2019-07-01', '2019-07-31', 'OldRandomForest')

## Testing

In [10]:
def get_num_fires(roi, start_date):
    image_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")
    e_d_i = (datetime.strptime(start_date, "%Y-%m-%d")+timedelta(days=16)).strftime("%Y-%m-%d")
    lsat_images = image_collection.filterBounds(roi.geometry()).\
                    filterDate(start_date, e_d_i).\
                    map(lambda image: image.clip(roi.geometry()))
    if lsat_images.size().getInfo() == 0:
        return -1
    fire_mask = getFireMaskOldRandomForest(lsat_images.mosaic())
    num_fire = fire_mask.reduceRegion(reducer=ee.Reducer.sum(),\
                                    geometry=roi.geometry(),scale = 30,\
                                    maxPixels = 1e8).getInfo()['F']
    return num_fire

In [11]:
bands_add = ['Evap_tavg', 'Rainf_f_tavg']
bands_avg = ['Tair_f_inst', 'Qair_f_inst', 'SoilTMP0_10cm_inst', 'Wind_f_inst']

def get_climate_data(roi, start_date):
    day_minus_1 = (datetime.strptime(start_date, "%Y-%m-%d")-timedelta(days=1)).strftime("%Y-%m-%d")
    day_minus_2 = (datetime.strptime(start_date, "%Y-%m-%d")-timedelta(days=2)).strftime("%Y-%m-%d")
    values = {}

    for ind, date in enumerate([start_date, day_minus_1, day_minus_2]):
        date_minus_1 = (datetime.strptime(date, "%Y-%m-%d")-timedelta(days=1)).strftime("%Y-%m-%d")
        gldas_images = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H")\
                        .filterBounds(roi.geometry())\
                        .filterDate(date_minus_1, date)
        image_avg = gldas_images.select(bands_avg).mean().clip(roi.geometry())
        mean_avg_values = image_avg.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi.geometry()).getInfo()
        mean_avg_values = {key+"_"+str(ind):val for key, val in mean_avg_values.items()}
        values.update(mean_avg_values)

        image_add = gldas_images.select(bands_add).sum().clip(roi.geometry())
        mean_add_values = image_add.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi.geometry()).getInfo()
        mean_add_values = {key+"_"+str(ind):val for key, val in mean_add_values.items()}
        values.update(mean_avg_values)
    return values


In [12]:
b = blocks.size().getInfo()
print(b)
blocks_list = blocks.toList(b)

631


In [13]:
def get_fire_info(lsat_images, roi):
    lsat_images = lsat_images.map(lambda image: image.set('area', image.geometry().area()))
    lsat_images = lsat_images.sort('area', False).first()
    fire_mask = getFireMaskOldRandomForest(lsat_images)
    num_fire = fire_mask.reduceRegion(reducer=ee.Reducer.sum(),\
                                    geometry=roi.geometry(),scale = 30,\
                                    maxPixels = 1e8)
    fire_feature = ee.Feature(None, num_fire)
    fire_feature = fire_feature.copyProperties(roi, ['Subdistric', 'aez_id', 'District N'])
    fire_feature = fire_feature.copyProperties(lsat_images, ['DATE_ACQUIRED'])
    return fire_feature.set("data", fire_feature.toDictionary(['F', 'Subdistric', 'aez_id', 'District N', 'DATE_ACQUIRED']))

def get_num_fires(roi, start_date):
    image_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")
    e_d_i = (datetime.strptime(start_date, "%Y-%m-%d")+timedelta(days=16)).strftime("%Y-%m-%d")
    lsat_images = image_collection.filterBounds(roi.geometry()).\
                    filterDate(start_date, e_d_i).\
                    map(lambda image: image.clip(roi.geometry()))
    return ee.Algorithms.If(lsat_images.size().eq(ee.Number(0)), ee.Feature(None, {}), get_fire_info(lsat_images, roi))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [31]:
def save_num_fires(block_list, date, chunks=50):
    itter = b//chunks
    df = pd.DataFrame()
    for i in range(itter,itter+1):
        #print("Doing for ", i*chunks,(i+1)*chunks)
        out = ee.FeatureCollection(blocks_list.slice(i*chunks,(i+1)*chunks)).map(lambda x: get_num_fires(x, date))
        data = out.aggregate_array('data').getInfo()
        df = pd.DataFrame.from_records(data)
        df.to_csv('/content/drive/MyDrive/Raman/forest_fires.csv', mode='a', header=False)
    #df = pd.concat([df, df_], ignore_index=True)

In [ ]:
start_date = '2019-05-01'
end_date = '2024-07-01'

start_date_o = datetime.strptime(start_date, "%Y-%m-%d")
end_date_o = datetime.strptime(end_date, "%Y-%m-%d")

while(start_date_o < end_date_o):
    date = start_date_o.strftime("%Y-%m-%d")
    print(date)
    save_num_fires(blocks_list, date)
    start_date_o = start_date_o+timedelta(days=16)

2019-05-01
2019-05-17
2019-06-02
2019-06-18
2019-07-04
2019-07-20
2019-08-05
2019-08-21
2019-09-06
2019-09-22
2019-10-08
2019-10-24
2019-11-09
2019-11-25
2019-12-11
2019-12-27
2020-01-12
2020-01-28
2020-02-13
2020-02-29
2020-03-16
2020-04-01
2020-04-17
2020-05-03
2020-05-19
2020-06-04
2020-06-20
2020-07-06
2020-07-22
2020-08-07
2020-08-23
2020-09-08
2020-09-24
2020-10-10
2020-10-26
2020-11-11
2020-11-27
2020-12-13
2020-12-29
2021-01-14
2021-01-30
2021-02-15
2021-03-03
2021-03-19
2021-04-04
2021-04-20
2021-05-06
2021-05-22
2021-06-07
2021-06-23
2021-07-09
2021-07-25
2021-08-10
2021-08-26
2021-09-11
2021-09-27
2021-10-13
2021-10-29
2021-11-14
2021-11-30
2021-12-16
2022-01-01
2022-01-17
2022-02-02
2022-02-18
2022-03-06
2022-03-22
2022-04-07
2022-04-23
2022-05-09
2022-05-25
2022-06-10
2022-06-26
2022-07-12
2022-07-28
2022-08-13
2022-08-29
2022-09-14
2022-09-30
2022-10-16
2022-11-01
2022-11-17
2022-12-03
2022-12-19
2023-01-04
2023-01-20
2023-02-05
2023-02-21
2023-03-09
2023-03-25
2023-04-10

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Raman/forest_fires_with_header.csv')

In [ ]:
len(df)

68820

In [ ]:
len(df[df['num_fire'] > 30])/len(df)

0.013833188026736413

In [ ]:
def get_indexes(blocks):
    indexes = blocks.map(lambda x: x.set('index_dis', x.toDictionary(['Subdistric','District N']))).aggregate_array('index_dis').getInfo()
    indexes = {thing["District N"] + "_" + thing["Subdistric"]:ind for ind,thing in enumerate(indexes)}
    return indexes

In [ ]:
def get_list(blocks):
    size = blocks.size().getInfo()
    bl_list = blocks.toList(size)
    block_list_python = []
    for i in range(size):
        block_list_python.append(ee.Feature(bl_list.get(i)))
    return block_list_python

def get_index_list(block_list, indexes):
    return ee.List([block_list[i] for i in indexes])

def get_climate_data(date, roi):
    day_minus_1 = (datetime.strptime(date, "%Y-%m-%d")-timedelta(days=1)).strftime("%Y-%m-%d")
    day_minus_2 = (datetime.strptime(date, "%Y-%m-%d")-timedelta(days=2)).strftime("%Y-%m-%d")
    day_minus_3 = (datetime.strptime(date, "%Y-%m-%d")-timedelta(days=3)).strftime("%Y-%m-%d")

    values = {
        "roi":roi.toDictionary(),
        "date":date
              }

    for ind, dates in enumerate([(day_minus_1, date), (day_minus_2, day_minus_1), (day_minus_3, day_minus_2)]):
        gldas_images = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H")\
                    .filterBounds(roi.geometry())\
                    .filterDate(dates[0], dates[1])

        image_avg = gldas_images.mean().clip(roi.geometry())
        image_avg = image_avg.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi.geometry())

        image_sum = gldas_images.sum().clip(roi.geometry())
        image_sum = image_sum.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi.geometry())

        image_max = gldas_images.max().clip(roi.geometry())
        image_max = image_max.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi.geometry())

        image_min = gldas_images.min().clip(roi.geometry())
        image_min = image_min.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi.geometry())

        values |= {
            "average_"+str(ind) : image_avg,
            "sum_"+str(ind) : image_sum,
            "max_"+str(ind) : image_max,
            "min_"+str(ind) : image_min

        }

    return ee.Feature(None).set("data", ee.Dictionary(values))

def flatten(data, fire):
    val_list = []
    for ind, data_bl in enumerate(data):
        keys = list(data_bl.keys())
        val = {"num_fire": fire[ind]}
        for k in keys:
            if k=="date":
                val["date"] = data_bl[k]
            else:
                keys_dic = list(data_bl[k].keys())
                for k_dic in keys_dic:
                    val[k_dic+"_"+k] = data_bl[k][k_dic]
        val_list.append(val)
    return val_list

In [ ]:
def save_climate_data(block_list, date, fire, file_ind, chunks=50):
    itter = block_list.size().getInfo()//chunks
    for i in range(0,itter+1):
        #print("Doing for ", i*chunks,(i+1)*chunks)
        data = ee.FeatureCollection(block_list.slice(i*chunks,(i+1)*chunks)).map(lambda x: get_climate_data(date, x))
        data = data.aggregate_array('data').getInfo()
        #print(len(data), len(fire[i*chunks:(i+1)*chunks]))
        d = flatten(data, fire[i*chunks:(i+1)*chunks])
        df = pd.DataFrame.from_records(d)
        if i==0:
            df.to_csv('/content/drive/MyDrive/Raman/climate_data_'+str(file_ind)+'.csv')
        else:
            df.to_csv('/content/drive/MyDrive/Raman/climate_data_'+str(file_ind)+'.csv', mode='a', header=False)

In [ ]:
i = 0
data_bl = df[['date','District N', 'Subdistric','num_fire']].drop_duplicates()
dates = data_bl.groupby('date').count().reset_index(drop=False)["date"].to_list()
block_list_python = get_list(blocks)
data_bl["ind"] = data_bl["District N"] + "_" + data_bl["Subdistric"]
indexes = get_indexes(blocks)
for ind, date in enumerate(dates):
    print("Date ", date, " Ind/Total ", ind,"/",len(dates))
    ind_ = [indexes[i] for i in data_bl[data_bl["date"]==date]["ind"].to_list()]
    blks = get_index_list(block_list_python, ind_)
    fire = data_bl[data_bl["date"]==date]["num_fire"].to_list()
    save_climate_data(ee.List(blks), date, fire, ind, chunks=50)

Date  2019-05-02  Ind/Total  0 / 660
Date  2019-05-04  Ind/Total  1 / 660
Date  2019-05-09  Ind/Total  2 / 660
Date  2019-05-11  Ind/Total  3 / 660
Date  2019-05-13  Ind/Total  4 / 660
Date  2019-05-16  Ind/Total  5 / 660
Date  2019-05-18  Ind/Total  6 / 660
Date  2019-05-20  Ind/Total  7 / 660
Date  2019-05-25  Ind/Total  8 / 660
Date  2019-05-27  Ind/Total  9 / 660
Date  2019-05-29  Ind/Total  10 / 660
Date  2019-06-01  Ind/Total  11 / 660
Date  2019-06-03  Ind/Total  12 / 660
Date  2019-06-05  Ind/Total  13 / 660
Date  2019-06-10  Ind/Total  14 / 660
Date  2019-06-12  Ind/Total  15 / 660
Date  2019-06-14  Ind/Total  16 / 660
Date  2019-06-17  Ind/Total  17 / 660
Date  2019-06-19  Ind/Total  18 / 660
Date  2019-06-21  Ind/Total  19 / 660
Date  2019-06-26  Ind/Total  20 / 660
Date  2019-06-28  Ind/Total  21 / 660
Date  2019-06-30  Ind/Total  22 / 660
Date  2019-07-12  Ind/Total  23 / 660
Date  2019-07-14  Ind/Total  24 / 660
Date  2019-07-16  Ind/Total  25 / 660
Date  2019-07-19  Ind/

In [ ]:
len(dates)

660

In [ ]:
blks.size().getInfo(), len(fire)

(68, 68)

In [ ]:
blks.slice(50,100).size()

In [ ]:
date
feature = get_climate_data(date, bk)
data = ee.FeatureCollection(blks).map(lambda x: get_climate_data(date, x))
data = data.aggregate_array('data').getInfo()
d = flatten(data)
df = pd.DataFrame.from_records(d)
df.to_csv('/content/drive/MyDrive/Raman/climate_data.csv')

In [47]:
gldas_images = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H")\
                    .filterBounds(ee.Feature(blocks_list.get(1)).geometry())\
                    .filterDate("2024-05-01", "2024-05-02")

In [48]:
gldas_images.first().clip(ee.Feature(blocks_list.get(1)).geometry())

In [54]:
lst = []
for i in range(blocks_list.size().getInfo()):
    print(i)
    lst.append(ee.Feature(blocks_list.get(1)).geometry().area().getInfo()/1000000)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


KeyboardInterrupt: 

In [16]:
area = blocks_list.map(lambda x: ee.Feature(x).geometry().area()).getInfo()

In [19]:
sum([i/1000000 for i in area])/631

290.819121353309